In [1]:

"""
GEIH 2023 - LIMPIEZA COMPLETA DE LOS 3 MODULOS
Ocupados + Fuerza de Trabajo + Hogar y Vivienda
"""

import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("GEIH 2023 - LIMPIEZA COMPLETA (3 MODULOS)")
print("="*80)

# =============================================================================
# DICCIONARIOS Y CONFIGURACION
# =============================================================================

RENAME_OCUPADOS = {
    "PERIODO": "periodo_encuesta", "MES": "mes_encuesta", "PER": "anio_encuesta",
    "DIRECTORIO": "id_hogar", "SECUENCIA_P": "id_persona", "ORDEN": "orden_encuesta",
    "AREA": "area_urbano_rural", "CLASE": "clase_territorial",
    "FEX_C18": "factor_expansion_hogar", "DPTO": "codigo_departamento",
    "P6426": "posicion_ocupacional", "INGLABO": "ingreso_laboral", 
    "OCI": "informalidad", "RAMA2D_R4": "rama_2_digitos", 
    "RAMA4D_R4": "rama_4_digitos", "OFICIO_C8": "oficio_codigo",
    "P6430": "numero_personas_hogar", "P3045S1": "tenencia_computador", 
    "P3045S2": "tenencia_internet", "P3363": "nivel_educativo", 
    "P9440": "alfabetismo", "P6500": "afiliacion_salud", 
    "P3364": "cotiza_pension", "P3364S1": "riesgos_laborales"
}

RENAME_FUERZA = {
    "DIRECTORIO": "id_hogar", "SECUENCIA_P": "id_persona",
    "PERIODO": "periodo_encuesta", "MES": "mes_encuesta", "PER": "anio_encuesta",
    "DPTO": "codigo_departamento", "AREA": "area_urbano_rural",
    "FEX_C18": "factor_expansion", "PET": "poblacion_edad_trabajar",
    "P6240": "condicion_actividad", "P6240S1": "ocupado", "P6240S2": "desocupado",
    "P6290": "posicion_ocupacional", "P6280": "rama_actividad",
    "P3362S1": "informal_sin_contrato", "P3362S2": "informal_sin_seguridad_social",
    "P3362S7": "informal_empresa_pequena", "P3362S7A1": "tamano_empresa",
    "P6300": "horas_trabajo_semana", "P6310": "segundo_trabajo",
    "P6320": "ingreso_laboral_principal", "P6340": "ingreso_total_laboral"
}

RENAME_HOGAR = {
    "DIRECTORIO": "id_hogar", "PERIODO": "periodo_encuesta", 
    "MES": "mes_encuesta", "PER": "anio_encuesta",
    "DPTO": "codigo_departamento", "AREA": "area_urbano_rural",
    "FEX_C18": "factor_expansion_hogar", "P70": "numero_personas_hogar",
    "P5000": "tenencia_computador", "P5010": "tenencia_internet",
    "P5050": "uso_internet_trabajo", "P5080": "uso_internet_comercio",
    "P5100": "nivel_educativo", "P5130": "alfabetismo",
    "P5222S1": "afiliacion_salud", "P5222S3": "cotiza_pension",
    "P5222S8": "beneficiario_transferencias", "P6008": "ingreso_total_hogar"
}

OUTPUT_DIR = 'LIMPIOS'
os.makedirs(OUTPUT_DIR, exist_ok=True)

MESES = ['01','02','03','04','05','06','07','08','09','10','11','12']

DEPTS = {
    5:'ANTIOQUIA', 8:'ATLANTICO', 11:'BOGOTA', 13:'BOLIVAR', 15:'BOYACA',
    17:'CALDAS', 19:'CAUCA', 20:'CESAR', 23:'CORDOBA', 25:'CUNDINAMARCA',
    27:'CHOCO', 41:'HUILA', 44:'LA GUAJIRA', 47:'MAGDALENA', 50:'META',
    52:'NARINO', 54:'NORTE DE SANTANDER', 63:'QUINDIO', 66:'RISARALDA',
    68:'SANTANDER', 70:'SUCRE', 73:'TOLIMA', 76:'VALLE DEL CAUCA'
}

# =============================================================================
# FUNCIONES AUXILIARES
# =============================================================================

def buscar_archivo(mes, tipo):
    patrones = []
    if tipo == 'Ocupados':
        patrones = [f"{mes}_Ocupados.CSV"]
    elif tipo == 'Fuerza':
        patrones = [f"{mes}_Fuerza de trabajo.CSV"]
    elif tipo == 'Hogar':
        patrones = [f"{mes}_Datos del hogar y la vivienda.CSV"]
    
    for patron in patrones:
        if os.path.exists("GEIH_2023/"+patron):
            return "GEIH_2023/"+patron
    
    if tipo == 'Ocupados':
        archivos = glob.glob(f"{mes}*[Oo]cupados*.CSV")
    elif tipo == 'Fuerza':
        archivos = glob.glob(f"{mes}*[Ff]uerza*.CSV")
    elif tipo == 'Hogar':
        archivos = glob.glob(f"{mes}*[Hh]ogar*.CSV") + glob.glob(f"{mes}*[Dd]atos*.CSV")
    
    return archivos[0] if archivos else None

def clasificar_sector(rama):
    if pd.isna(rama):
        return 'Desconocido'
    rama_str = str(int(rama)).zfill(4) if isinstance(rama, (int, float)) else str(rama).zfill(4)
    rama_2d = rama_str[:2]
    
    if rama_2d in ['01','02','03']:
        return 'Agricultura'
    elif '10' <= rama_2d <= '33':
        return 'Manufactura'
    elif '45' <= rama_2d <= '47':
        return 'Comercio'
    elif rama_2d in ['55','56']:
        return 'Alojamiento y restaurantes'
    elif '49' <= rama_2d <= '53':
        return 'Transporte'
    elif '94' <= rama_2d <= '96':
        return 'Servicios personales'
    else:
        return 'Otros servicios'

def cargar_modulo(tipo, rename_dict, meses):
    dfs = []
    for mes in meses:
        archivo = buscar_archivo(mes, tipo)
        if archivo:
            print(f"  [{mes}] {archivo}...", end=" ")
            try:
                df = pd.read_csv(archivo, sep=';', encoding='latin-1', low_memory=False)
                df['MES_CARGA'] = mes
                dfs.append(df)
                print(f"OK ({len(df):,})")
            except Exception as e:
                print(f"ERROR: {e}")
    
    if dfs:
        resultado = pd.concat(dfs, ignore_index=True)
        resultado = resultado.rename(columns={k:v for k,v in rename_dict.items() if k in resultado.columns})
        print(f"  TOTAL: {len(resultado):,} filas")
        return resultado
    return None

# =============================================================================
# PASO 1: CARGAR MODULOS
# =============================================================================

print("\n" + "="*80)
print("PASO 1: CARGA Y CONCATENACION")
print("="*80)

print("\n[1/3] OCUPADOS")
ocupados = cargar_modulo('Ocupados', RENAME_OCUPADOS, MESES)

print("\n[2/3] FUERZA DE TRABAJO")
fuerza = cargar_modulo('Fuerza', RENAME_FUERZA, MESES)

print("\n[3/3] HOGAR Y VIVIENDA")
hogar = cargar_modulo('Hogar', RENAME_HOGAR, MESES)

# =============================================================================
# PASO 2: LIMPIEZA OCUPADOS
# =============================================================================

print("\n" + "="*80)
print("PASO 2: LIMPIEZA - OCUPADOS")
print("="*80)

if ocupados is not None:
    print(f"\nDimensiones iniciales: {ocupados.shape}")
    
    # Convertir a numerico
    for col in ['ingreso_laboral', 'numero_personas_hogar']:
        if col in ocupados.columns:
            ocupados[col] = pd.to_numeric(ocupados[col], errors='coerce')
    
    # Imputar
    if 'ingreso_laboral' in ocupados.columns and ocupados['ingreso_laboral'].notna().sum() > 0:
        ocupados['ingreso_laboral'] = ocupados['ingreso_laboral'].fillna(ocupados['ingreso_laboral'].median())
    
    # Binarias a 0
    for col in ['tenencia_computador', 'tenencia_internet', 'afiliacion_salud', 'cotiza_pension']:
        if col in ocupados.columns:
            ocupados[col] = ocupados[col].fillna(0).astype(int)
    
    # Duplicados
    ocupados = ocupados.drop_duplicates()
    
    # Variables derivadas
    if 'codigo_departamento' in ocupados.columns:
        ocupados['nombre_departamento'] = ocupados['codigo_departamento'].map(DEPTS)
    
    if 'area_urbano_rural' in ocupados.columns:
        ocupados['area_urbana'] = (ocupados['area_urbano_rural'] == 1).astype(int)
    
    if 'rama_4_digitos' in ocupados.columns:
        ocupados['sector_economico'] = ocupados['rama_4_digitos'].apply(clasificar_sector)
    
    # Seleccionar columnas relevantes
    cols_ocup = ['id_hogar', 'id_persona', 'periodo_encuesta', 'mes_encuesta', 'MES_CARGA',
                 'codigo_departamento', 'nombre_departamento', 'area_urbana',
                 'posicion_ocupacional', 'informalidad', 'sector_economico', 
                 'rama_2_digitos', 'rama_4_digitos', 'ingreso_laboral',
                 'numero_personas_hogar', 'tenencia_internet', 'nivel_educativo',
                 'afiliacion_salud', 'cotiza_pension', 'factor_expansion_hogar']
    
    cols_ocup_final = [c for c in cols_ocup if c in ocupados.columns]
    ocupados_limpio = ocupados[cols_ocup_final].copy()
    
    # Guardar
    ocupados_limpio.to_csv(f'{OUTPUT_DIR}/ocupados_2023_limpio.csv', index=False, encoding='utf-8-sig')
    size = os.path.getsize(f'{OUTPUT_DIR}/ocupados_2023_limpio.csv') / (1024*1024)
    print(f"\nGuardado: ocupados_2023_limpio.csv")
    print(f"  Dimensiones: {ocupados_limpio.shape}")
    print(f"  Tamaño: {size:.1f} MB")

# =============================================================================
# PASO 3: LIMPIEZA FUERZA DE TRABAJO
# =============================================================================

print("\n" + "="*80)
print("PASO 3: LIMPIEZA - FUERZA DE TRABAJO")
print("="*80)

if fuerza is not None:
    print(f"\nDimensiones iniciales: {fuerza.shape}")
    
    # Convertir a numerico
    for col in ['ingreso_laboral_principal', 'ingreso_total_laboral', 'horas_trabajo_semana']:
        if col in fuerza.columns:
            fuerza[col] = pd.to_numeric(fuerza[col], errors='coerce')
    
    # Imputar binarias (convertir a numerico primero)
    for col in ['ocupado', 'desocupado', 'segundo_trabajo']:
        if col in fuerza.columns:
            fuerza[col] = pd.to_numeric(fuerza[col], errors='coerce')
            fuerza[col] = fuerza[col].fillna(0).astype(int)
    
    # Duplicados
    fuerza = fuerza.drop_duplicates()
    
    # Variables derivadas
    if 'codigo_departamento' in fuerza.columns:
        fuerza['nombre_departamento'] = fuerza['codigo_departamento'].map(DEPTS)
    
    if 'area_urbano_rural' in fuerza.columns:
        fuerza['area_urbana'] = (fuerza['area_urbano_rural'] == 1).astype(int)
    
    # Seleccionar columnas relevantes
    cols_fuerza = ['id_hogar', 'id_persona', 'periodo_encuesta', 'mes_encuesta', 'MES_CARGA',
                   'codigo_departamento', 'nombre_departamento', 'area_urbana',
                   'condicion_actividad', 'ocupado', 'desocupado',
                   'posicion_ocupacional', 'rama_actividad',
                   'informal_sin_contrato', 'informal_sin_seguridad_social',
                   'horas_trabajo_semana', 'segundo_trabajo',
                   'ingreso_laboral_principal', 'ingreso_total_laboral',
                   'factor_expansion']
    
    cols_fuerza_final = [c for c in cols_fuerza if c in fuerza.columns]
    fuerza_limpio = fuerza[cols_fuerza_final].copy()
    
    # Guardar
    fuerza_limpio.to_csv(f'{OUTPUT_DIR}/fuerza_2023_limpio.csv', index=False, encoding='utf-8-sig')
    size = os.path.getsize(f'{OUTPUT_DIR}/fuerza_2023_limpio.csv') / (1024*1024)
    print(f"\nGuardado: fuerza_2023_limpio.csv")
    print(f"  Dimensiones: {fuerza_limpio.shape}")
    print(f"  Tamaño: {size:.1f} MB")

# =============================================================================
# PASO 4: LIMPIEZA HOGAR Y VIVIENDA
# =============================================================================

print("\n" + "="*80)
print("PASO 4: LIMPIEZA - HOGAR Y VIVIENDA")
print("="*80)

if hogar is not None:
    print(f"\nDimensiones iniciales: {hogar.shape}")
    
    # Convertir a numerico
    if 'ingreso_total_hogar' in hogar.columns:
        hogar['ingreso_total_hogar'] = pd.to_numeric(hogar['ingreso_total_hogar'], errors='coerce')
    
    if 'numero_personas_hogar' in hogar.columns:
        hogar['numero_personas_hogar'] = pd.to_numeric(hogar['numero_personas_hogar'], errors='coerce')
    
    # Imputar binarias (convertir a numerico primero)
    for col in ['tenencia_computador', 'tenencia_internet', 'afiliacion_salud', 
                'cotiza_pension', 'beneficiario_transferencias']:
        if col in hogar.columns:
            hogar[col] = pd.to_numeric(hogar[col], errors='coerce')
            hogar[col] = hogar[col].fillna(0).astype(int)
    
    # Duplicados
    hogar = hogar.drop_duplicates()
    
    # Variables derivadas
    if 'codigo_departamento' in hogar.columns:
        hogar['nombre_departamento'] = hogar['codigo_departamento'].map(DEPTS)
    
    if 'area_urbano_rural' in hogar.columns:
        hogar['area_urbana'] = (hogar['area_urbano_rural'] == 1).astype(int)
    
    # Seleccionar columnas relevantes
    cols_hogar = ['id_hogar', 'periodo_encuesta', 'mes_encuesta', 'MES_CARGA',
                  'codigo_departamento', 'nombre_departamento', 'area_urbana',
                  'numero_personas_hogar', 'tenencia_computador', 'tenencia_internet',
                  'uso_internet_trabajo', 'uso_internet_comercio',
                  'nivel_educativo', 'alfabetismo', 'afiliacion_salud',
                  'cotiza_pension', 'beneficiario_transferencias',
                  'ingreso_total_hogar', 'factor_expansion_hogar']
    
    cols_hogar_final = [c for c in cols_hogar if c in hogar.columns]
    hogar_limpio = hogar[cols_hogar_final].copy()
    
    # Guardar
    hogar_limpio.to_csv(f'{OUTPUT_DIR}/hogar_2023_limpio.csv', index=False, encoding='utf-8-sig')
    size = os.path.getsize(f'{OUTPUT_DIR}/hogar_2023_limpio.csv') / (1024*1024)
    print(f"\nGuardado: hogar_2023_limpio.csv")
    print(f"  Dimensiones: {hogar_limpio.shape}")
    print(f"  Tamaño: {size:.1f} MB")

# =============================================================================
# PASO 5: EMPRENDEDORES INFORMALES
# =============================================================================

print("\n" + "="*80)
print("PASO 5: FILTRAR EMPRENDEDORES INFORMALES")
print("="*80)

if ocupados is not None and 'posicion_ocupacional' in ocupados.columns:
    mask_posicion = ocupados['posicion_ocupacional'].isin([3, 4])
    
    if 'informalidad' in ocupados.columns:
        mask_informal = ocupados['informalidad'] == 1
        emprendedores = ocupados[mask_posicion & mask_informal].copy()
    else:
        emprendedores = ocupados[mask_posicion].copy()
    
    print(f"\nEmprendedores identificados: {len(emprendedores):,}")
    
    # Guardar
    cols_emp = ['id_hogar', 'id_persona', 'codigo_departamento', 'nombre_departamento',
                'sector_economico', 'posicion_ocupacional', 'informalidad',
                'ingreso_laboral', 'rama_4_digitos', 'tenencia_internet',
                'nivel_educativo', 'afiliacion_salud', 'MES_CARGA']
    
    cols_emp_final = [c for c in cols_emp if c in emprendedores.columns]
    emprendedores_limpio = emprendedores[cols_emp_final].copy()
    
    emprendedores_limpio.to_csv(f'{OUTPUT_DIR}/emprendedores_informales_2023.csv',
                                 index=False, encoding='utf-8-sig')
    size = os.path.getsize(f'{OUTPUT_DIR}/emprendedores_informales_2023.csv') / (1024*1024)
    print(f"\nGuardado: emprendedores_informales_2023.csv")
    print(f"  Dimensiones: {emprendedores_limpio.shape}")
    print(f"  Tamaño: {size:.1f} MB")
    
    # Agregacion departamento-sector
    if 'sector_economico' in emprendedores.columns and 'ingreso_laboral' in emprendedores.columns:
        contexto = emprendedores.groupby(['codigo_departamento', 'sector_economico']).agg({
            'ingreso_laboral': ['median', 'mean', 'count']
        }).reset_index()
        
        contexto.columns = ['codigo_departamento', 'sector_economico',
                           'ingreso_mediano', 'ingreso_promedio', 'num_emprendedores']
        
        contexto.to_csv(f'{OUTPUT_DIR}/contexto_laboral_dept_sector.csv',
                       index=False, encoding='utf-8-sig')
        print(f"\nGuardado: contexto_laboral_dept_sector.csv ({contexto.shape})")

# =============================================================================
# RESUMEN FINAL
# =============================================================================

print("\n" + "="*80)
print("RESUMEN FINAL")
print("="*80)

print(f"\nArchivos generados en '{OUTPUT_DIR}/':")
archivos = [
    'ocupados_2023_limpio.csv',
    'fuerza_2023_limpio.csv',
    'hogar_2023_limpio.csv',
    'emprendedores_informales_2023.csv',
    'contexto_laboral_dept_sector.csv'
]

for archivo in archivos:
    ruta = f'{OUTPUT_DIR}/{archivo}'
    if os.path.exists(ruta):
        size = os.path.getsize(ruta) / (1024*1024)
        print(f"  {archivo} ({size:.1f} MB)")
    else:
        print(f"  {archivo} (NO GENERADO)")

print("\n" + "="*80)
print("LIMPIEZA COMPLETADA - 3 MODULOS LISTOS")
print("="*80)


GEIH 2023 - LIMPIEZA COMPLETA (3 MODULOS)

PASO 1: CARGA Y CONCATENACION

[1/3] OCUPADOS
  [01] GEIH_2023/01_Ocupados.CSV... OK (29,695)
  [02] GEIH_2023/02_Ocupados.CSV... OK (30,726)
  [03] GEIH_2023/03_Ocupados.CSV... OK (31,009)
  [04] GEIH_2023/04_Ocupados.CSV... OK (30,654)
  [05] GEIH_2023/05_Ocupados.CSV... OK (30,517)
  [06] GEIH_2023/06_Ocupados.CSV... OK (30,535)
  [07] GEIH_2023/07_Ocupados.CSV... OK (31,018)
  [08] GEIH_2023/08_Ocupados.CSV... OK (31,086)
  [09] GEIH_2023/09_Ocupados.CSV... OK (30,586)
  [10] GEIH_2023/10_Ocupados.CSV... OK (29,797)
  [11] GEIH_2023/11_Ocupados.CSV... OK (30,479)
  [12] GEIH_2023/12_Ocupados.CSV... OK (29,717)
  TOTAL: 365,819 filas

[2/3] FUERZA DE TRABAJO
  [01] GEIH_2023/01_Fuerza de trabajo.CSV... OK (57,364)
  [02] GEIH_2023/02_Fuerza de trabajo.CSV... OK (58,035)
  [03] GEIH_2023/03_Fuerza de trabajo.CSV... OK (57,404)
  [04] GEIH_2023/04_Fuerza de trabajo.CSV... OK (56,543)
  [05] GEIH_2023/05_Fuerza de trabajo.CSV... OK (56,214)
  